# UPS Plus (EP-0136) バッテリーモニター

52Pi UPS Plus から I2C 経由でバッテリー情報を取得します。

- **0x17**: UPS MCU (バッテリー電圧, 容量, 充電ポート検出)
- **0x40**: INA219 供給電力モニター (RPi 消費電力)
- **0x45**: INA219 バッテリーモニター (充放電電流)

In [ ]:
from ina219 import INA219
import smbus2

bus = smbus2.SMBus(1)
MCU_ADDR = 0x17

ina_supply = INA219(0.00725, busnum=1, address=0x40)
ina_supply.configure()

ina_batt = INA219(0.005, busnum=1, address=0x45)
ina_batt.configure()

# UPS MCU レジスタアドレス (バイトアドレス)
REG_MCU_VOLTAGE = 0x01       # MCU 内部電圧 (mV)
REG_BATT_VOLTAGE = 0x05      # バッテリー端子電圧 (mV)
REG_USB_C_VOLTAGE = 0x07     # USB-C 充電ポート電圧 (mV)
REG_MICRO_USB_VOLTAGE = 0x09 # MicroUSB 充電ポート電圧 (mV)
REG_BATT_TEMP = 0x0B         # バッテリー温度 (℃)
REG_BATT_CAPACITY = 0x13     # バッテリー残量 (%)

print("UPS Plus 接続 OK")

In [ ]:
# === 全ステータス取得 ===
print("=== Supply (RPi) ===")
print(f"  電圧:   {ina_supply.voltage():.2f} V")
print(f"  電流:   {ina_supply.current():.1f} mA")
print(f"  電力:   {ina_supply.power():.1f} mW")

print("\n=== Battery ===")
print(f"  電圧:   {ina_batt.voltage():.2f} V")
batt_i = ina_batt.current()
print(f"  電流:   {batt_i:.1f} mA ({'充電中' if batt_i < 0 else '放電中'})")
print(f"  電力:   {ina_batt.power():.1f} mW")

print("\n=== MCU ===")
batt_mv = bus.read_word_data(MCU_ADDR, REG_BATT_VOLTAGE)
batt_cap = bus.read_word_data(MCU_ADDR, REG_BATT_CAPACITY)
usb_c_mv = bus.read_word_data(MCU_ADDR, REG_USB_C_VOLTAGE)
micro_usb_mv = bus.read_word_data(MCU_ADDR, REG_MICRO_USB_VOLTAGE)
batt_temp = bus.read_word_data(MCU_ADDR, REG_BATT_TEMP)
print(f"  バッテリー電圧:   {batt_mv} mV")
print(f"  バッテリー温度:   {batt_temp} ℃")
print(f"  残量:             {batt_cap} %")
print(f"  USB-C 電圧:       {usb_c_mv} mV")
print(f"  MicroUSB 電圧:    {micro_usb_mv} mV")

In [ ]:
# === バッテリー電圧の推移を記録 ===
import time
import matplotlib.pyplot as plt

DURATION = 60   # 秒
INTERVAL = 2    # 秒

timestamps = []
voltages = []
currents = []

print(f"{DURATION}秒間モニタリング開始 ({INTERVAL}秒間隔)")
start = time.time()
while time.time() - start < DURATION:
    t = time.time() - start
    v = ina_batt.voltage()
    i = ina_batt.current()
    timestamps.append(t)
    voltages.append(v)
    currents.append(i)
    print(f"  {t:5.1f}s  V={v:.2f}V  I={i:.0f}mA", end="\r")
    time.sleep(INTERVAL)

print(f"\n{len(timestamps)} サンプル取得完了")

In [ ]:
# === グラフ表示 ===
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

ax1.plot(timestamps, voltages, "b-")
ax1.set_ylabel("Battery Voltage (V)")
ax1.grid(True)

ax2.plot(timestamps, currents, "r-")
ax2.set_ylabel("Battery Current (mA)")
ax2.set_xlabel("Time (s)")
ax2.axhline(y=0, color="gray", linestyle="--", linewidth=0.5)
ax2.grid(True)

fig.suptitle("UPS Plus Battery Monitor")
plt.tight_layout()
plt.show()

In [ ]:
bus.close()
print("切断完了")